In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [3]:
import pandas as pd
import numpy as np

import re
import os
import math

from sklearn.ensemble import RandomForestRegressor

from pandas.api.types import is_string_dtype, is_numeric_dtype


In [6]:
train_df = pd.read_csv('Train.csv', low_memory=False, parse_dates=['saledate'])
valid_df = pd.read_csv('Valid.csv', low_memory=False, parse_dates=['saledate'])

In [7]:
def display_all(df):
    with pd.option_context('display.max_rows',1000, 'display.max_columns',100):
        display(df)


In [8]:
display_all(train_df.tail().T)

,401120,401121,401122,401123,401124
SalesID,6333336,6333337,6333338,6333341,6333342
SalePrice,10500,11000,11500,9000,7750
MachineID,1840702,1830472,1887659,1903570,1926965
ModelID,21439,21439,21439,21435,21435
datasource,149,149,149,149,149
auctioneerID,1,1,1,2,2
YearMade,2005,2005,2005,2005,2005
MachineHoursCurrentMeter,NaN,NaN,NaN,NaN,NaN
UsageBand,NaN,NaN,NaN,NaN,NaN
saledate,2011-11-02 00:00:00,2011-11-02 00:00:00,2011-11-02 00:00:00,2011-10-25 00:00:00,2011-10-25 00:00:00


In [9]:
display_all(train_df.describe(include='all').T)

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
SalesID,401125,NaN,NaN,NaN,NaT,NaT,1.91971e+06,909021,1.13925e+06,1.41837e+06,1.63942e+06,2.24271e+06,6.33334e+06
SalePrice,401125,NaN,NaN,NaN,NaT,NaT,31099.7,23036.9,4750,14500,24000,40000,142000
MachineID,401125,NaN,NaN,NaN,NaT,NaT,1.2179e+06,440992,0,1.0887e+06,1.27949e+06,1.46807e+06,2.48633e+06
ModelID,401125,NaN,NaN,NaN,NaT,NaT,6889.7,6221.78,28,3259,4604,8724,37198
datasource,401125,NaN,NaN,NaN,NaT,NaT,134.666,8.96224,121,132,132,136,172
auctioneerID,380989,NaN,NaN,NaN,NaT,NaT,6.55604,16.9768,0,1,2,4,99
YearMade,401125,NaN,NaN,NaN,NaT,NaT,1899.16,291.797,1000,1985,1995,2000,2013
MachineHoursCurrentMeter,142765,NaN,NaN,NaN,NaT,NaT,3457.96,27590.3,0,0,0,3025,2.4833e+06
UsageBand,69639,3,Medium,33985,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
saledate,401125,3919,2009-02-16 00:00:00,1932,1989-01-17,2011-12-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
train_df.SalePrice = np.log(train_df.SalePrice)


In [11]:
train_df.SalePrice[:5]

0    11.097410
1    10.950807
2     9.210340
3    10.558414
4     9.305651
Name: SalePrice, dtype: float64

In [12]:
def add_datepart(df, fldname, drop=True):
    fld = df[fldname]
    
    # if fld is not of type datetime convert it to datetime
    if not np.issubdtype(fld.dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
        
    targ_pre = re.sub('[Dd]ate$','',fldname)
    
    for n in ('Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear', 'Is_month_end', 
             'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start'):
        df[targ_pre+n] = getattr(fld.dt, n.lower())
        
    df[targ_pre+'Elapsed'] = fld.astype(np.int64) // 10**9
    if drop:
        df.drop(fldname, axis=1, inplace=True)


In [13]:
add_datepart(train_df, 'saledate')
add_datepart(valid_df, 'saledate')
train_df.saleYear.head()

0    2006
1    2004
2    2004
3    2011
4    2009
Name: saleYear, dtype: int64

In [14]:
def train_cats(df):
    for n, c in df.items():
        if is_string_dtype(c): df[n] = c.astype('category').cat.as_ordered()
            
def apply_cats(df, train):
    for n, c in df.items():
        if (n in train.columns) and (train[n].dtype.name == 'category'):
            df[n] = c.astype('category').cat.as_ordered()
            df[n].cat.set_categories(train[n].cat.categories, ordered=True, inplace=True)

In [15]:
train_cats(train_df)
apply_cats(valid_df, train_df)

In [16]:
train_df.UsageBand.cat.categories

Index(['High', 'Low', 'Medium'], dtype='object')

In [17]:
train_df.UsageBand.cat.set_categories(['High', 'Medium', 'Low'], ordered=True, inplace=True)
valid_df.UsageBand.cat.set_categories(['High', 'Medium', 'Low'], ordered=True, inplace=True)

In [18]:
display_all(train_df.isnull().sum())
print('--------------------------------------')
display_all(valid_df.isnull().sum())

SalesID                          0
SalePrice                        0
MachineID                        0
ModelID                          0
datasource                       0
auctioneerID                 20136
YearMade                         0
MachineHoursCurrentMeter    258360
UsageBand                   331486
fiModelDesc                      0
fiBaseModel                      0
fiSecondaryDesc             137191
fiModelSeries               344217
fiModelDescriptor           329206
ProductSize                 210775
fiProductClassDesc               0
state                            0
ProductGroup                     0
ProductGroupDesc                 0
Drive_System                296764
Enclosure                      325
Forks                       209048
Pad_Type                    321991
Ride_Control                252519
Stick                       321991
Transmission                217895
Turbocharged                321991
Blade_Extension             375906
Blade_Width         

--------------------------------------


SalesID                         0
MachineID                       0
ModelID                         0
datasource                      0
auctioneerID                    0
YearMade                        0
MachineHoursCurrentMeter     6834
UsageBand                    7542
fiModelDesc                    88
fiBaseModel                    19
fiSecondaryDesc              3538
fiModelSeries                9815
fiModelDescriptor            8677
ProductSize                  5830
fiProductClassDesc              0
state                           0
ProductGroup                    0
ProductGroupDesc                0
Drive_System                 8847
Enclosure                       9
Forks                        5935
Pad_Type                     9611
Ride_Control                 7451
Stick                        9611
Transmission                 6796
Turbocharged                 9611
Blade_Extension             10809
Blade_Width                 10809
Enclosure_Type              10809
Engine_Horsepo

In [19]:
def fix_missing(df, col, name):
    if is_numeric_dtype(col):
        if pd.isnull(col).sum():
            #df[name+'_na'] = pd.isnull(col)
            filler = col.median()
            df[name] = col.fillna(filler)
            #na_dict[name] = filler

In [20]:
def numericalize(df, col, name):
    if not is_numeric_dtype(col):
        df[name] = col.cat.codes + 1

In [21]:
def proc_df(df, y_fld=None):
    
    if y_fld is None: y = None
    else:
        if not is_numeric_dtype(df[y_fld]) : df[y_fld] = df[y_fld].cat.codes
        y = df[y_fld].values
        df.drop(y_fld, axis=1, inplace=True)
    
    for n,c in df.items(): fix_missing(df, c, n)
    for n,c in df.items(): numericalize(df, c, n)
        
    df = pd.get_dummies(df, dummy_na=True)
    
    return [df,y]

In [22]:
train, y = proc_df(train_df, 'SalePrice')
test, temp = proc_df(valid_df)

In [23]:
def split_vals(a, n): return a[:n].copy(), a[n:].copy()

n_valid = 12000
n_train = len(train) - n_valid

X_train, X_valid = split_vals(train, n_train)
y_train, y_valid = split_vals(y, n_train)
X_train.shape, y_train.shape, X_valid.shape

((389125, 64), (389125,), (12000, 64))

In [24]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
           m.score(X_train, y_train), m.score(X_valid, y_valid)]
    print(res)

In [25]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

C:\Users\acer\Anaconda3\envs\sonika\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Wall time: 1min 30s
[0.09028802221073266, 0.24662019758497364, 0.9829629222394961, 0.8913811359156507]


In [26]:
y_pred = m.predict(test)

In [28]:
Submission = pd.DataFrame({'SalesID':valid_df.SalesID, 'SalePrice':y_pred})
Submission.to_csv('Submission.csv', index=False)